In [1]:
import re
import string
import torch
from torch import nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from torch import optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
device = "cpu"

In [3]:
df = pd.read_csv('twitter_cleaned.csv')
df = df.dropna()

In [4]:
df.head()

,label,message,category
0,neutral,prenatal move to wednesday at pm starting toni...,1
1,positive,happy nd birthday to prince george i cant beli...,2
2,positive,do not be afraid to be saint be open to the lo...,2
3,neutral,dst is saturday nightsunday morning got ta be ...,1
4,negative,sony reward app is like a lot of yo female sin...,0


In [5]:
x_train, x_test, y_train, y_test = train_test_split(
    df['message'], df['category'], test_size=.2, stratify=df['label'])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(39740,) (9935,) (39740,) (9935,)


In [6]:
vectorizer = TfidfVectorizer(max_features=2000)
# vectorizer = TfidfVectorizer(max_features=1500, stop_words='english')

# Learn vocabulary from training texts and vectorize training texts.
x_train = vectorizer.fit_transform(x_train)

# Vectorize test texts.
x_test = vectorizer.transform(x_test)

In [7]:
x_train = torch.tensor(scipy.sparse.csr_matrix.todense(x_train)).float()
x_test = torch.tensor(scipy.sparse.csr_matrix.todense(x_test)).float()

In [8]:
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [9]:
def topk_encoding(nd_array):
    """
    Function to flatten the predicted category
    """
    
    predictions = nd_array
    
    ps = torch.exp(predictions)
    top_p, top_class  = ps.topk(1, dim=1)
    

    return top_class

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.hidden_layer_1 = nn.Linear(x_train.shape[1], 64) # input to first hidden layer
        self.output_layer = nn.Linear(64, self.out_dim)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.hidden_layer_1(x)
        x = self.activation(x)
        x = self.dropout(x)
        
        y = self.output_layer(x)
        y = self.activation(y)
        y = self.softmax(y)
        
        return y

In [11]:
model = NeuralNetwork(x_train.shape[1], df['category'].nunique())
# model = NeuralNetwork(x_train.shape[1], 5)


# Define the loss
criterion = nn.NLLLoss()


# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.Adam(model.parameters(), lr=0.002)

#setting up scheduler
# scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 10)

In [12]:
%%time
train_losses = []
test_losses = []
test_accuracies = []

epochs = 200
for e in range(epochs):
    optimizer.zero_grad()

    output = model.forward(x_train) #Forward pass, get the logits
    loss = criterion(output, y_train) # Calculate the loss with the logits and the labels
    loss.backward()
    train_loss = loss.item()
    train_losses.append(train_loss)
    
    optimizer.step()

    # Turn off gradients for validation, saves memory and computations
    with torch.no_grad():
        model.eval()
        log_ps = model.forward(x_test)
        test_loss = criterion(log_ps, y_test)
        test_losses.append(test_loss)

        ps = torch.exp(log_ps)
        top_p, top_class  = ps.topk(1, dim=1)
        equals = top_class == y_test.view(*top_class.shape)
        test_accuracy = torch.mean(equals.float())
        test_accuracies.append(test_accuracy)

    model.train()

    print(f"Epoch: {e+1}/{epochs}.. ",
          f"Training Loss: {train_loss:.3f}.. ",
          f"Test Loss: {test_loss:.3f}.. ",
          f"Test Accuracy: {test_accuracy:.3f}")
#     scheduler.step(test_loss/len(y_test))

Epoch: 1/200..  Training Loss: 1.099..  Test Loss: 1.099..  Test Accuracy: 0.163
Epoch: 2/200..  Training Loss: 1.099..  Test Loss: 1.098..  Test Accuracy: 0.303
Epoch: 3/200..  Training Loss: 1.098..  Test Loss: 1.097..  Test Accuracy: 0.440
Epoch: 4/200..  Training Loss: 1.097..  Test Loss: 1.095..  Test Accuracy: 0.448
Epoch: 5/200..  Training Loss: 1.095..  Test Loss: 1.093..  Test Accuracy: 0.448
Epoch: 6/200..  Training Loss: 1.093..  Test Loss: 1.091..  Test Accuracy: 0.448
Epoch: 7/200..  Training Loss: 1.091..  Test Loss: 1.089..  Test Accuracy: 0.448
Epoch: 8/200..  Training Loss: 1.089..  Test Loss: 1.087..  Test Accuracy: 0.448
Epoch: 9/200..  Training Loss: 1.087..  Test Loss: 1.085..  Test Accuracy: 0.448
Epoch: 10/200..  Training Loss: 1.084..  Test Loss: 1.082..  Test Accuracy: 0.448
Epoch: 11/200..  Training Loss: 1.082..  Test Loss: 1.080..  Test Accuracy: 0.448
Epoch: 12/200..  Training Loss: 1.079..  Test Loss: 1.077..  Test Accuracy: 0.448
Epoch: 13/200..  Training

Epoch: 103/200..  Training Loss: 0.955..  Test Loss: 1.002..  Test Accuracy: 0.429
Epoch: 104/200..  Training Loss: 0.955..  Test Loss: 1.002..  Test Accuracy: 0.428
Epoch: 105/200..  Training Loss: 0.954..  Test Loss: 1.002..  Test Accuracy: 0.428
Epoch: 106/200..  Training Loss: 0.954..  Test Loss: 1.002..  Test Accuracy: 0.428
Epoch: 107/200..  Training Loss: 0.954..  Test Loss: 1.002..  Test Accuracy: 0.428
Epoch: 108/200..  Training Loss: 0.953..  Test Loss: 1.002..  Test Accuracy: 0.428
Epoch: 109/200..  Training Loss: 0.952..  Test Loss: 1.002..  Test Accuracy: 0.428
Epoch: 110/200..  Training Loss: 0.952..  Test Loss: 1.002..  Test Accuracy: 0.427
Epoch: 111/200..  Training Loss: 0.952..  Test Loss: 1.002..  Test Accuracy: 0.427
Epoch: 112/200..  Training Loss: 0.951..  Test Loss: 1.002..  Test Accuracy: 0.427
Epoch: 113/200..  Training Loss: 0.951..  Test Loss: 1.002..  Test Accuracy: 0.427
Epoch: 114/200..  Training Loss: 0.951..  Test Loss: 1.002..  Test Accuracy: 0.427
Epoc

In [13]:
preds = model.forward(x_test)
preds = topk_encoding(preds)
preds[1]

tensor([1])

In [14]:
acc_score = accuracy_score(y_test, preds)
acc_score

0.4120785103170609

In [15]:
f1_score = f1_score(y_test, preds, average='macro')
f1_score

0.3114127848675836

### ML Pipeline

In [16]:
def to_lower(message):
    result = message.lower()
    return result

def remove_num(message):
    result = re.sub(r'\d+','',message)
    return result

def contractions(message):
     result = re.sub(r"won't", "will not",message)
     result = re.sub(r"would't", "would not",message)
     result = re.sub(r"could't", "could not",message)
     result = re.sub(r"\'d", " would",message)
     result = re.sub(r"can\'t", "can not",message)
     result = re.sub(r"n\'t", " not", message)
     result = re.sub(r"\'re", " are", message)
     result = re.sub(r"\'s", " is", message)
     result = re.sub(r"\'ll", " will", message)
     result = re.sub(r"\'t", " not", message)
     result = re.sub(r"\'ve", " have", message)
     result = re.sub(r"\'m", " am", message)
     return result
    
def remove_punctuation(message):
    result = message.translate(str.maketrans(dict.fromkeys(string.punctuation)))
    return result

def remove_whitespace(message):
    result = message.strip()
    result = re.sub(' +',' ',message)
    return result

def replace_newline(message):
    result = message.replace('\n','')
    return result

def data_cleanup(message):
    cleaning_utils = [to_lower, remove_num, contractions, remove_punctuation, remove_whitespace, replace_newline]
    for util in cleaning_utils:
        message = util(message)
    return message


In [17]:
def input_vectorizer(message):
    """
    Function to predict the category of inputted message
    """
    
    cleaned_message = pd.Series(message).apply(lambda x: data_cleanup(x))
    vec = vectorizer.transform(pd.Series(cleaned_message))
    vec = torch.tensor(scipy.sparse.csr_matrix.todense(vec)).float()
    preds = model.forward(vec)
    category = topk_encoding(preds).detach().cpu().numpy()
    
    return int(category[0])

In [18]:
a = input_vectorizer("fly to the sky to go back after a year on september.")
a


1

In [19]:
b = input_vectorizer("the item is not good, waste of money")
b


0